In [ ]:
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


In [2]:
import pandas as pd
from nltk.corpus import stopwords
import re
import spacy
import matplotlib.pyplot as plt
nlp = spacy.load('en_core_web_sm')
df = pd.read_excel(r"C:\Users\follm\Documents\coding\music-blog-analyzer\scraping\Pitchfork Albums\page 1 to 100001 data scraping_2020-01-10.xlsx")

In [3]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))
def clean_description(x):
    x = eval(x)
    text = ' '.join([pp for pp in x if not re.search('^Buy: |Pitchfork may .* affiliate', pp)]).lower()
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    text = ' '.join(tok.lemma_ for tok in nlp(text))
    return text

df['body_clean'] = df['body'].apply(lambda x: clean_description(x))

In [32]:
beg = 20
end = 7
df.rating = df.rating.str[beg:-end].astype(float)
df['genre_split'] = df['genre'].apply(lambda x: [] if pd.isnull(x) else eval(x))
unique_genres = set([genre for list_of_genres in df['genre_split'].tolist() for genre in list_of_genres])
for genre in unique_genres:
    df[genre] = df['genre_split'].apply(lambda x: genre in x)

In [46]:
print(df['body_clean'].apply(lambda x: len(x)).describe())
VOCAB_SIZE = 50000
MAX_REVIEW_LENGTH = 2000
#VOCAB_SIZE = 50000
#MAX_REVIEW_LENGTH = 3500

count    22058.000000
mean      2896.554991
std       1062.265322
min          0.000000
25%       2250.000000
50%       2695.000000
75%       3268.750000
max      15721.000000
Name: body_clean, dtype: float64


In [51]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(num_words=VOCAB_SIZE, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)

In [48]:
tokenizer.fit_on_texts(df['body_clean'].values)
word_index = tokenizer.word_index
print('found %s unique tokens' % len(word_index))

found 277552 unique tokens


In [53]:
X = tokenizer.texts_to_sequences(df['body_clean'].values)
X = pad_sequences(X, maxlen=MAX_REVIEW_LENGTH)
print('Shape of data tensor:', X.shape)


Shape of data tensor: (22058, 2000)


In [54]:
Y = pd.get_dummies(df['genre_split'].str[0]).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (22058, 9)


In [55]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)


(19852, 2000) (19852, 9)
(2206, 2000) (2206, 9)


In [58]:
from keras.models import Sequential
from keras.layers import Dense, Activation, SpatialDropout1D, Embedding, LSTM
from keras.callbacks.callbacks import EarlyStopping

In [60]:
EMBEDDING_DIM = 100

model = Sequential()
model.add(Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(Y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])


C:\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 17866 samples, validate on 1986 samples
Epoch 1/5
17866/17866 [==============================] - 4117s 230ms/step - loss: 1.5674 - accuracy: 0.3534 - val_loss: 1.5842 - val_accuracy: 0.3535
Epoch 2/5
17866/17866 [==============================] - 3866s 216ms/step - loss: 1.5484 - accuracy: 0.3555 - val_loss: 1.5848 - val_accuracy: 0.3535
Epoch 3/5
17866/17866 [==============================] - 3705s 207ms/step - loss: 1.5486 - accuracy: 0.3555 - val_loss: 1.5922 - val_accuracy: 0.3535
Epoch 4/5
17866/17866 [==============================] - 3857s 216ms/step - loss: 1.5489 - accuracy: 0.3556 - val_loss: 1.5900 - val_accuracy: 0.3535


In [9]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb

# set parameters:
max_features = 5000
maxlen = 400
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2

print('Loading data...')
(imdb_x_train, imdb_y_train), (imdb_x_test, imdb_y_test) = imdb.load_data(num_words=max_features)
print(len(imdb_x_train), 'train sequences')
print(len(imdb_x_test), 'test sequences')


Loading data...
17465344/17464789 [==============================] - 2s 0us/step
25000 train sequences
25000 test sequences


In [27]:
plot(df)


400

In [ ]:

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))
